# Eigenvalues and Linear Transformations

**SIIEA Quantum Engineering Curriculum**
- **Curriculum Days:** Days 85-112
- **License:** CC BY-NC-SA 4.0 | Siiea Innovations, LLC

---

In [ ]:
# Hardware detection — adapts simulations to your machine
import sys, os
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath("__file__")), ".."))
try:
    from hardware_config import HARDWARE, get_max_qubits
    print(f"Hardware: {HARDWARE['chip']} | {HARDWARE['memory_gb']} GB | Profile: {HARDWARE['profile']}")
    print(f"Max qubits: {get_max_qubits('safe')} (safe) / {get_max_qubits('max')} (max)")
except ImportError:
    print("hardware_config.py not found — using defaults")
    print("Run setup.sh from the repo root to generate it")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from numpy.linalg import eig, svd, det, inv, norm, qr
%matplotlib inline

# Publication-quality plot defaults
plt.rcParams.update({
    'figure.figsize': (8, 6),
    'font.size': 12,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'lines.linewidth': 2,
    'figure.dpi': 100,
})
print("Imports loaded. Ready for linear algebra explorations.")

## Why Linear Algebra IS the Language of Quantum Mechanics

Every quantum state is a **vector** in a complex Hilbert space. Every observable
is a **Hermitian matrix**. Every time-evolution is a **unitary transformation**.
Understanding eigenvalues, eigenvectors, and matrix decompositions is not merely
preparation for quantum mechanics --- it *is* quantum mechanics in its most
fundamental mathematical form.

**Key correspondences:**

| Linear Algebra Concept | Quantum Mechanics Meaning |
|------------------------|--------------------------|
| Vector $\|v\rangle$ | Quantum state |
| Matrix $A$ | Observable or operator |
| Eigenvalue $\lambda$ | Measurement outcome |
| Eigenvector $\|\lambda\rangle$ | State after measurement |
| Unitary matrix $U$ | Time evolution |
| Inner product $\langle u \| v \rangle$ | Probability amplitude |

In this notebook we build fluency with all of these concepts computationally.

## 1. Matrix Operations Review

A matrix $A \in \mathbb{R}^{m \times n}$ is a rectangular array of numbers.
Key operations:

- **Multiplication:** $(AB)_{ij} = \sum_k A_{ik} B_{kj}$
- **Transpose:** $(A^T)_{ij} = A_{ji}$
- **Determinant** (square matrices): $\det(A)$ encodes volume scaling
- **Inverse:** $A^{-1}$ exists iff $\det(A) \neq 0$

$$\det\begin{pmatrix} a & b \\ c & d \end{pmatrix} = ad - bc$$

In [ ]:
# --- Matrix Operations: multiplication, transpose, determinant, inverse ---

A = np.array([[2, 1],
              [1, 3]])
B = np.array([[1, -1],
              [0,  2]])

print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)

# Matrix multiplication (two equivalent ways)
AB = A @ B
print("\nA @ B (matrix product):")
print(AB)

# Transpose
print("\nA^T (transpose):")
print(A.T)

# Determinant
det_A = det(A)
print(f"\ndet(A) = {det_A:.4f}")
print(f"det(B) = {det(B):.4f}")
print(f"det(AB) = {det(AB):.4f}")
print(f"det(A) * det(B) = {det_A * det(B):.4f}  (should match det(AB))")

# Inverse
A_inv = inv(A)
print("\nA^{-1}:")
print(A_inv)
print("\nVerification A @ A^{-1} = I:")
print(np.round(A @ A_inv, 10))

## 2. Eigenvalue Decomposition

An eigenvector $\mathbf{v}$ of matrix $A$ satisfies:

$$A\mathbf{v} = \lambda \mathbf{v}$$

where $\lambda$ is the corresponding **eigenvalue**. Geometrically, $A$ stretches
$\mathbf{v}$ by factor $\lambda$ without changing its direction.

For a diagonalizable matrix: $A = P \Lambda P^{-1}$ where $\Lambda$ is diagonal
with eigenvalues and $P$ contains eigenvectors as columns.

**In QM:** measuring an observable $\hat{A}$ on eigenstate $|\lambda\rangle$ always
yields the eigenvalue $\lambda$. The spectral decomposition
$\hat{A} = \sum_i \lambda_i |\lambda_i\rangle\langle\lambda_i|$ is foundational.

In [ ]:
# --- Eigenvalue decomposition with verification ---

# Symmetric matrix (guaranteed real eigenvalues)
A = np.array([[4, 1],
              [1, 3]])
print("Matrix A:")
print(A)

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = eig(A)
print(f"\nEigenvalues: {eigenvalues}")
print(f"\nEigenvectors (as columns):")
print(eigenvectors)

# Verification: A @ v = lambda * v for each eigenpair
print("\n--- Verification: A v = lambda v ---")
for i in range(len(eigenvalues)):
    lam = eigenvalues[i]
    v = eigenvectors[:, i]
    Av = A @ v
    lam_v = lam * v
    residual = norm(Av - lam_v)
    print(f"  lambda_{i} = {lam:.4f}")
    print(f"    A @ v_{i}     = {Av}")
    print(f"    lambda * v_{i} = {lam_v}")
    print(f"    ||Av - lv||   = {residual:.2e} {'PASS' if residual < 1e-10 else 'FAIL'}")

# Reconstruction: A = P Lambda P^{-1}
P = eigenvectors
Lambda = np.diag(eigenvalues)
A_reconstructed = P @ Lambda @ inv(P)
print(f"\nReconstruction error ||A - P Lambda P^{{-1}}|| = {norm(A - A_reconstructed):.2e}")

## 3. Visualizing 2D Linear Transformations

A $2 \times 2$ matrix $A$ maps every point $(x, y)$ to a new point $(x', y')$.
The unit circle maps to an **ellipse** whose semi-axes are the **singular values**
of $A$, and whose principal directions are given by the singular vectors.

The eigenvectors of $A$ are the directions that remain unchanged (only scaled)
under the transformation.

In [ ]:
# --- Visualize how a matrix transforms the unit circle ---

A = np.array([[2.0, 1.0],
              [0.5, 1.5]])

# Generate unit circle
theta = np.linspace(0, 2 * np.pi, 300)
circle = np.array([np.cos(theta), np.sin(theta)])  # shape (2, 300)

# Apply transformation
transformed = A @ circle  # shape (2, 300)

# Get eigenvalues/vectors for annotation
vals, vecs = eig(A)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Original unit circle
axes[0].plot(circle[0], circle[1], 'b-', linewidth=2, label='Unit circle')
axes[0].set_xlim(-3, 3)
axes[0].set_ylim(-3, 3)
axes[0].set_aspect('equal')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(0, color='k', linewidth=0.5)
axes[0].axvline(0, color='k', linewidth=0.5)
axes[0].set_title('Original: Unit Circle')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')

# Transformed ellipse with eigenvectors
axes[1].plot(transformed[0], transformed[1], 'r-', linewidth=2, label='Transformed')
for i in range(2):
    v = vecs[:, i].real
    lam = vals[i].real
    # Plot eigenvector scaled by eigenvalue
    axes[1].annotate('', xy=lam * v, xytext=[0, 0],
                     arrowprops=dict(arrowstyle='->', color=f'C{i+2}', lw=2.5))
    axes[1].annotate(f'$\lambda_{i+1}={lam:.2f}$',
                     xy=lam * v * 1.15, fontsize=11, color=f'C{i+2}', weight='bold')

axes[1].set_xlim(-4, 4)
axes[1].set_ylim(-4, 4)
axes[1].set_aspect('equal')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(0, color='k', linewidth=0.5)
axes[1].axvline(0, color='k', linewidth=0.5)
axes[1].set_title(f'After transformation by A (det={det(A):.2f})')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')

plt.suptitle('Linear Transformation: Unit Circle to Ellipse', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

print(f"Eigenvalues: {vals}")
print(f"Area scaling factor (det A): {det(A):.4f}")

## 4. Singular Value Decomposition (SVD)

Every matrix $A \in \mathbb{R}^{m \times n}$ admits a decomposition:

$$A = U \Sigma V^T$$

where:
- $U \in \mathbb{R}^{m \times m}$ is orthogonal (left singular vectors)
- $\Sigma \in \mathbb{R}^{m \times n}$ is diagonal with singular values $\sigma_1 \geq \sigma_2 \geq \cdots \geq 0$
- $V \in \mathbb{R}^{n \times n}$ is orthogonal (right singular vectors)

**Applications:**
- Low-rank approximation (image compression)
- Principal Component Analysis
- Pseudoinverse computation

**QM Connection:** Schmidt decomposition of bipartite quantum states is the SVD
applied to the coefficient matrix, revealing entanglement structure.

In [ ]:
# --- SVD Image Compression Demo ---
# Create a synthetic "image" (grayscale gradient + shapes)

np.random.seed(42)
N = 128

# Create a test image with geometric features
img = np.zeros((N, N))
# Gradient background
for i in range(N):
    for j in range(N):
        img[i, j] = 0.3 * (i + j) / (2 * N)
# Rectangle
img[20:50, 30:90] = 0.9
# Circle
yy, xx = np.ogrid[:N, :N]
circle_mask = (xx - 80)**2 + (yy - 80)**2 < 20**2
img[circle_mask] = 0.7
# Diagonal stripe
for i in range(N):
    j_start = max(0, i - 5)
    j_end = min(N, i + 5)
    img[i, j_start:j_end] = np.maximum(img[i, j_start:j_end], 0.5)

# Perform SVD
U, sigma, Vt = svd(img, full_matrices=False)
print(f"Image shape: {img.shape}")
print(f"Number of singular values: {len(sigma)}")
print(f"Top 10 singular values: {sigma[:10].round(2)}")

# Reconstruct with different ranks
ranks = [1, 5, 10, 20, 50, 128]
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for ax, k in zip(axes.flat, ranks):
    # Rank-k approximation
    img_k = U[:, :k] @ np.diag(sigma[:k]) @ Vt[:k, :]
    # Compression ratio: original = N*N, compressed = k*(N+N+1)
    compression = (k * (N + N + 1)) / (N * N) * 100
    error = norm(img - img_k) / norm(img) * 100

    ax.imshow(img_k, cmap='gray', vmin=0, vmax=1)
    ax.set_title(f'Rank {k}\n{compression:.1f}% of data, {error:.1f}% error')
    ax.axis('off')

plt.suptitle('SVD Image Compression: Rank-k Approximations', fontsize=15)
plt.tight_layout()
plt.show()

# Plot singular value spectrum
fig, ax = plt.subplots(figsize=(10, 4))
ax.semilogy(sigma, 'o-', markersize=3)
ax.set_xlabel('Index $k$')
ax.set_ylabel('Singular value $\sigma_k$')
ax.set_title('Singular Value Spectrum (log scale)')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
print(f"\nSigma_1 / Sigma_N ratio (condition number): {sigma[0]/sigma[-1]:.2f}")

## 5. Quantum Gates as Matrices

Quantum gates are **unitary matrices** acting on qubit state vectors.
The most fundamental single-qubit gates are the **Pauli matrices** and
the **Hadamard gate**:

$$X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}, \quad
Y = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix}, \quad
Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$$

$$H = \frac{1}{\sqrt{2}}\begin{pmatrix} 1 & 1 \\ 1 & -1 \end{pmatrix}$$

**Properties:**
- All Pauli matrices are **Hermitian** ($X = X^\dagger$) and **unitary** ($XX^\dagger = I$)
- Eigenvalues of Pauli matrices are $\pm 1$ (measurement outcomes)
- $H$ creates superposition: $H|0\rangle = \frac{|0\rangle + |1\rangle}{\sqrt{2}}$

In [ ]:
# --- Quantum Gates: Pauli matrices and Hadamard ---

# Define the standard qubit basis
ket_0 = np.array([1, 0], dtype=complex)  # |0>
ket_1 = np.array([0, 1], dtype=complex)  # |1>

# Pauli matrices
I2 = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)    # Bit flip
Y = np.array([[0, -1j], [1j, 0]], dtype=complex) # Bit + phase flip
Z = np.array([[1, 0], [0, -1]], dtype=complex)    # Phase flip
H = np.array([[1, 1], [1, -1]], dtype=complex) / np.sqrt(2)  # Hadamard

gates = {'I': I2, 'X': X, 'Y': Y, 'Z': Z, 'H': H}

print("=== Quantum Gate Properties ===\n")
for name, G in gates.items():
    vals, vecs = eig(G)
    is_hermitian = np.allclose(G, G.conj().T)
    is_unitary = np.allclose(G @ G.conj().T, I2)

    print(f"--- {name} Gate ---")
    print(f"  Matrix:\n{G}")
    print(f"  Eigenvalues:  {np.round(vals, 4)}")
    print(f"  Hermitian:    {is_hermitian}")
    print(f"  Unitary:      {is_unitary}")
    print(f"  det = {det(G):.4f}")
    print()

# Demonstrate Hadamard creating superposition
print("=== Hadamard on |0> ===")
psi = H @ ket_0
print(f"H|0> = {psi}  =  (|0> + |1>)/sqrt(2)")
print(f"Probabilities: |<0|psi>|^2 = {abs(psi[0])**2:.4f}, |<1|psi>|^2 = {abs(psi[1])**2:.4f}")

# Show Pauli algebra: XY = iZ, etc.
print("\n=== Pauli Algebra ===")
print(f"XY = iZ?  {np.allclose(X @ Y, 1j * Z)}")
print(f"YZ = iX?  {np.allclose(Y @ Z, 1j * X)}")
print(f"ZX = iY?  {np.allclose(Z @ X, 1j * Y)}")
print(f"X^2 = I?  {np.allclose(X @ X, I2)}")

## 6. Gram-Schmidt Orthogonalization

Given a set of linearly independent vectors $\{\mathbf{v}_1, \dots, \mathbf{v}_n\}$,
the Gram-Schmidt process produces an **orthonormal** set $\{\mathbf{e}_1, \dots, \mathbf{e}_n\}$:

$$\mathbf{u}_k = \mathbf{v}_k - \sum_{j=1}^{k-1} \frac{\langle \mathbf{v}_k, \mathbf{e}_j \rangle}{\langle \mathbf{e}_j, \mathbf{e}_j \rangle} \mathbf{e}_j, \qquad \mathbf{e}_k = \frac{\mathbf{u}_k}{\|\mathbf{u}_k\|}$$

**QM Connection:** Orthonormal bases are fundamental. Measurement postulate requires
orthogonal projectors. Every quantum computation begins with choosing an orthonormal basis.

In [ ]:
# --- Gram-Schmidt Orthogonalization from scratch ---

def gram_schmidt(V):
    """
    Gram-Schmidt orthogonalization.

    Parameters
    ----------
    V : ndarray, shape (n, k)
        k column vectors of dimension n (must be linearly independent).

    Returns
    -------
    Q : ndarray, shape (n, k)
        Orthonormal columns spanning the same space as V.
    """
    n, k = V.shape
    Q = np.zeros_like(V, dtype=float)

    for j in range(k):
        # Start with the original vector
        u = V[:, j].astype(float).copy()

        # Subtract projections onto all previous orthonormal vectors
        for i in range(j):
            u -= np.dot(Q[:, i], V[:, j]) * Q[:, i]

        # Normalize
        u_norm = norm(u)
        if u_norm < 1e-12:
            raise ValueError(f"Vectors are linearly dependent at index {j}")
        Q[:, j] = u / u_norm

    return Q


# Test with 3D vectors
V = np.array([
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1]
], dtype=float).T  # Each column is a vector

print("Original vectors (as columns):")
print(V)

Q = gram_schmidt(V)
print("\nOrthonormalized vectors (as columns):")
print(np.round(Q, 6))

# Verify orthonormality: Q^T Q should be identity
QTQ = Q.T @ Q
print("\nQ^T Q (should be identity):")
print(np.round(QTQ, 10))
print(f"\nMax deviation from identity: {np.max(np.abs(QTQ - np.eye(3))):.2e}")

# Compare with NumPy's QR decomposition (which uses Gram-Schmidt internally)
Q_np, R_np = qr(V)
# qr may flip signs; compare absolute values of columns
print("\n--- Comparison with np.linalg.qr ---")
for i in range(3):
    # Columns may differ by a sign
    match = np.allclose(Q[:, i], Q_np[:, i]) or np.allclose(Q[:, i], -Q_np[:, i])
    print(f"  Column {i} matches QR: {match}")

In [ ]:
# --- Visualize eigenvalue spectrum of a random symmetric matrix ---

np.random.seed(0)
N_dim = 50
# Create random symmetric matrix (guaranteed real eigenvalues)
M = np.random.randn(N_dim, N_dim)
M_sym = (M + M.T) / 2  # Symmetrize

eigenvalues_sym = np.sort(np.linalg.eigvalsh(M_sym))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of eigenvalues (approaches Wigner semicircle for large N)
axes[0].hist(eigenvalues_sym, bins=20, edgecolor='black', alpha=0.7, color='steelblue')
axes[0].set_xlabel('Eigenvalue $\lambda$')
axes[0].set_ylabel('Count')
axes[0].set_title(f'Eigenvalue Distribution ({N_dim}x{N_dim} Random Symmetric)')
axes[0].axvline(0, color='red', linestyle='--', alpha=0.5)
axes[0].grid(True, alpha=0.3)

# Eigenvalue staircase (cumulative distribution)
axes[1].step(eigenvalues_sym, np.arange(1, N_dim + 1) / N_dim, where='post',
             linewidth=2, color='darkgreen')
axes[1].set_xlabel('Eigenvalue $\lambda$')
axes[1].set_ylabel('Cumulative fraction')
axes[1].set_title('Eigenvalue Cumulative Distribution')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Eigenvalue range: [{eigenvalues_sym[0]:.3f}, {eigenvalues_sym[-1]:.3f}]")
print(f"Trace = sum of eigenvalues = {np.trace(M_sym):.4f} vs {np.sum(eigenvalues_sym):.4f}")

## Summary

| Concept | Key Formula | NumPy Function |
|---------|-------------|----------------|
| Eigendecomposition | $Av = \lambda v$ | `np.linalg.eig(A)` |
| SVD | $A = U\Sigma V^T$ | `np.linalg.svd(A)` |
| Determinant | $\det(A)$ | `np.linalg.det(A)` |
| Gram-Schmidt | $e_k = u_k / \|u_k\|$ | `np.linalg.qr(A)` |
| Matrix exponential | $e^{A}$ | `scipy.linalg.expm(A)` |

**Key Takeaways:**
1. Eigenvalues determine how a matrix scales along special directions
2. SVD generalizes eigendecomposition to non-square matrices
3. Quantum gates are unitary matrices; their eigenvalues have unit modulus
4. Pauli matrices form a basis for all 2x2 Hermitian matrices
5. Gram-Schmidt converts any basis to an orthonormal one

---
*Next: Month 05 explores complex vector spaces, Hermitian/unitary matrices, and the spectral theorem.*